Você pode rodar este notebook localmente ou no Colab. Para abrir diretamente no Colab, basta clicar no botão abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap01/cap01-main.ipynb) 

# Capítulo 1 - Explorando os ambientes

Nosso objetivo nesta aula é apenas motivar para o estudo de **Aprendizagem por Reforço** (*Reinforcement Learning* ou *RL*).

Para isso, vamos explorar os ambientes (simuladores de tarefas) oferecidos no pacote **`gym`**, originalmente criado pela OpenAI para praticar e pesquisar RL. 

A `gym` inclui diversos ambientes interessantes. E na internet, você encontra outros ambientes compatíveis com o `gym`.

***Atenção***: *este pacote foi descontinuado em favor do pacote `gymnasium`, mas vou continuar com o `gym` pela simplicidade e para aguardar o `gymnasium` ficar mais estável.*

## Configurações Iniciais

Para instalar e importar pacotes e configurar algumas coisas...

In [20]:
from IPython.display import clear_output

# for saving videos
!apt-get install ffmpeg freeglut3-dev xvfb

clear_output()

In [21]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [24]:
## TODO: mover para util.notebook
from base64 import b64encode
from IPython.display import HTML, display

from pathlib import Path

def display_videos_from_path(videos_folder='', prefix=''):
    """
    Adapted from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(videos_folder).glob("{}*.mp4".format(prefix)):
        video_b64 = b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                    </video>'''.format(mp4, video_b64.decode('ascii')))
    display(HTML(data="<br>".join(html)))


In [ ]:
!pip install swig
!pip install gym[box2d]==0.23.1 

In [7]:
import gym
import time

'0.23.1'

## 1 - Testando Alguns Ambientes

### 1.1 Uso Básico

Abaixo, mostramos um uso básico. Mostramos como passar *ações* para o ambiente e como ler as *observações* e *recompensas* vindas dele.

In [ ]:
# descomente apenas a linha do ambiente que lhe interessa
env = gym.make("MountainCar-v0")
#env = gym.make("Taxi-v3")
#env = gym.make("CartPole-v1")
#env = gym.make("Pendulum-v1")
#env = gym.make("LunarLander-v2")

# reinicia um episodio no ambiente, e retorna a observação inicial
obs = env.reset()

# variável para indicar se o episódio acabou
done = False

while not done:
    # para renderizar (se você estiver rodando no PC)
    #env.render()

    # escolhe uma ação aleatória
    # neste ponto, você pode usar um algoritmo qualquer para escolher a ação
    action = env.action_space.sample()
    print("ACTION:", action)

    # aplica a ação no ambiente
    (obs, r, done, _) = env.step(action)
    print("NEW OBS:", obs)
    print("REWARD :", r)
    print("-")

    # espera um pouco, para mostrar os resultados mais lentamente
    time.sleep(0.2)

# encerra o ambiente, principalmente se usar renderização
env.close()

O código abaixo calcula o **retorno** (soma das recompensas) ao fim do episódio. 

In [22]:
env = gym.make("MountainCar-v0")
#env = gym.make("Taxi-v3")
#env = gym.make("CartPole-v1")
#env = gym.make("Pendulum-v1")
#env = gym.make("LunarLander-v2")

obs = env.reset()
done = False
sum_rewards = 0.0

while not done:
    #env.render()
    action = env.action_space.sample()

    obs, reward, done, info = env.step(action)
    sum_rewards += reward

    #time.sleep(0.1)

env.close()
print("Retorno (soma das recompensas):", sum_rewards)

Retorno (soma das recompensas): -200.0


### 1.2 Gravando e Exibindo Vídeos

No exemplo a seguir, mostramos como **gravar vídeos** da execução do ambiente, usando o wrapper `RecordVideo`. 
- Esta classe encapsula o ambiente original e pode ser usada como se fosse o próprio ambiente.
- Internamente, ela executa o ambiente original e salva vídeos com a renderização dos passos. 
- Os vídeos são salvos em arquivos MP4 no diretório informado. Este exemplo salva apenas 1.

In [ ]:
video_path = '/content/videos'
rec_env = gym.wrappers.RecordVideo(env, video_path)

obs = rec_env.reset()
done = False

while not done:
    action = rec_env.action_space.sample()
    (obs, r, done, _) = rec_env.step(action)

rec_env.close()


Para exibir os vídeos em um notebook como este, usamos a função `display_videos_from_path()`.

In [27]:
display_videos_from_path(video_path)

## 2 - Solução Manual

Vamos tentar resolver o ambiente **Mountain Car** criando um algoritmo "manual" para escolher a ação.

Vejas as informações sobre este ambiente aqui: https://gymnasium.farama.org/environments/classic_control/mountain_car/ . 

Segue um resumo:

- O episódio **termina**:
  - quando o carro chega na *bandeira* no lado direito
  - ou quando atingir o máximo de 200 passos sem chegar lá
- Cada **observação** deste ambiente é uma lista com estas duas informações:
  - *índice 0* - posição no carro no eixo *x*
  - *índice 1* - velocidade do carro 
- As **ações** possíveis são:
  - *0* - acelerar para a esquerda
  - *1* - deixar livre
  - *2* - acelerar para a direita
- A cada passo, a **recompensa** é -1, para incentivar o agente a encerrar a tarefa rapidamente



***Agora, tente criar sua solução!*** 

Você saberá que resolveu o ambiente se o retorno do episódio for acima de -200 (por exemplo: -157).

In [13]:
env = gym.make("MountainCar-v0")

obs = env.reset()
done = False
sum_rewards = 0.0

while not done:
    #env.render()
    
    # CRIE AQUI uma forma de escolher a ação, entre as três possíveis ações
    action = 0

    obs, reward, done, _ = env.step(action)
    sum_rewards += reward
    
    time.sleep(0.01)

print("Retorno:", sum_rewards)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Retorno: -200.0


Você consegue fazer uma boa solução sem precisar deixar o carrinho livre (`action=1`).

Para chegar a uma boa solução, pense no seguinte:
- em quais situações faz sentido acelerar para a esquerda (`action=0`)?
- em quais situações faz sentido acelerar para a direita (`action=2`)?

## 3 - Informações Sobre o Ambiente

Os ambientes podem ter observações contínuas (geralmente como listas de valores float) ou discretas (geralmente um único inteiro).

Também as ações podem ser contínuas ou discretas. Cada ambiente do `gym` informa:
- qual o seu **espaço de observações** (atributo `env.observation_space`)
- e o seu **espaço de ações** (atributo `env.action_space`)

Criamos a função abaixo para ler um espaço (de ações ou observações) e detalhar suas informações

In [14]:
def print_space_info(space):
    if isinstance(space, gym.spaces.Discrete):
        print("   - quantidade de valores:", space.n)
    elif isinstance(space, gym.spaces.Box):
        print("   - formato/shape:", space.shape)
        print("   - valores mínimos (por item):", space.low)
        print("   - valores máximos (por item):", space.high)


In [15]:
#env = gym.make("MountainCar-v0")
#env = gym.make("Taxi-v3")
#env = gym.make("CartPole-v1")
env = gym.make("Pendulum-v1")
#env = gym.make("LunarLander-v2")

print("INFORMAÇÕES SOBRE O AMBIENTE", env)
print()
print("=> OBSERVATION SPACE:", env.observation_space)
print_space_info(env.observation_space)

print()
print("=> ACTION SPACE:", env.action_space)
print_space_info(env.action_space)
print()


INFORMAÇÕES SOBRE O AMBIENTE <TimeLimit<OrderEnforcing<PendulumEnv<Pendulum-v1>>>>

=> OBSERVATION SPACE: Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)
   - formato/shape: (3,)
   - valores mínimos (por item): [-1. -1. -8.]
   - valores máximos (por item): [1. 1. 8.]

=> ACTION SPACE: Box(-2.0, 2.0, (1,), float32)
   - formato/shape: (1,)
   - valores mínimos (por item): [-2.]
   - valores máximos (por item): [2.]



## 4 - Definindo Wrappers

In [3]:
class PunishEarlyStop(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        # if ended because the pole fell down
        if done and ('TimeLimit.truncated' not in info):
            reward = -100
        return obs, reward, done, info


env = gym.make("CartPole-v1")
env = PunishEarlyStop(env)

obs = env.reset()
done = False
sum_rewards = 0.0

while not done:
    action = env.action_space.sample()

    obs, reward, done, info = env.step(action)
    sum_rewards += reward

env.close()
print("Recompensa total:", sum_rewards)

Recompensa total: -82.0
